In [2]:
from pathlib import Path

from pyspark.sql.session import SparkSession
from pyspark.sql import DataFrame

MAX_MEMORY = "8g"
spark = (
    SparkSession.builder.master("local[*]")
    .appName("TripAnaliysis")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.excutor.memory", MAX_MEMORY)
    .config("spark.driver.memory", MAX_MEMORY)
    .config("spark.sql.caseSensitive", "true")
    .getOrCreate()
)

24/05/10 18:03:51 WARN Utils: Your hostname, limhaneul-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.109 instead (on interface en0)
24/05/10 18:03:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/05/10 18:03:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import os
from pathlib import Path
from pyspark.sql import DataFrame
from pyspark.sql.session import SparkSession


class FileLoadInParquet:
    def __init__(self, year: int, taxi_type: str) -> None:
        self.year = year
        self.taxi_type = taxi_type
    
    def location(self) -> list[str]:
        return str(Path(os.getcwd()).parent.joinpath(f"data/{self.taxi_type}/{self.year}"))



In [4]:
from pyspark.sql.functions import col, lit
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, IntegerType

# 주어진 스키마에 따라 PySpark의 데이터 타입으로 변환
schema = StructType([
    StructField("vendor_id", StringType(), True),
    StructField("pickup_datetime", StringType(), True),
    StructField("dropoff_datetime", StringType(), True),
    StructField("passenger_count", LongType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("pickup_longitude", DoubleType(), True),
    StructField("pickup_latitude", DoubleType(), True),
    StructField("rate_code", StringType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("dropoff_longitude", DoubleType(), True),
    StructField("dropoff_latitude", DoubleType(), True),
    StructField("payment_type", LongType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("surcharge", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("congestion_surcharge", DoubleType(), True),
    StructField("airport_fee", DoubleType(), True)
])

# 2009 ~ 2024년 별로 스키마가 다름 정규화 작업 실행 

In [6]:
from pyspark.sql import Column

def read_parquet_data(year: int, spark: SparkSession) -> DataFrame:
    # 파케이 파일 경로
    data: str = FileLoadInParquet(year, "YellowTaxi").location()
    return (
        spark.read
        .parquet(f"file:///{data}/*"))

def checking_count(data: DataFrame) -> DataFrame:
    sum_null: list[Column] = [F.sum(col(c).isNull().cast(IntegerType())).alias(c) for c in data.columns]
    null_checking: None = data.agg(*sum_null).cache()

    return null_checking

for year in range(2009, 2025):
    # data = read_parquet_data(year, spark).printSchema()
    # print(data)
    a = read_parquet_data(year, spark).show(10)
    # checking_count(read_parquet_data(year, spark)).toPandas().to_csv(f"{year}_null_checking.csv", index=False, index_label=False)

+-----------+--------------------+---------------------+---------------+------------------+------------------+---------+---------+-----------------+------------------+---------+------------+-----------------+---------+-------+-------+---------+---------+---------+
|vendor_name|Trip_Pickup_DateTime|Trip_Dropoff_DateTime|Passenger_Count|     Trip_Distance|         Start_Lon|Start_Lat|Rate_Code|store_and_forward|           End_Lon|  End_Lat|Payment_Type|         Fare_Amt|surcharge|mta_tax|Tip_Amt|Tolls_Amt|Total_Amt|Rate_code|
+-----------+--------------------+---------------------+---------------+------------------+------------------+---------+---------+-----------------+------------------+---------+------------+-----------------+---------+-------+-------+---------+---------+---------+
|        VTS| 2009-01-04 02:52:00|  2009-01-04 03:02:00|              1|              2.63|        -73.991957|40.721567|     null|             null|        -73.993803|40.695922|        CASH|              8

Py4JJavaError: An error occurred while calling o117.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 39.0 failed 1 times, most recent failure: Lost task 0.0 in stage 39.0 (TID 39) (192.168.0.109 executor driver): org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file file:///Users/imhaneul/Documents/project/SparkTaxiInsights/sparkAnaliysis/data/YellowTaxi/2018/yellow_tripdata_2018-03.parquet. Column: [congestion_surcharge], Expected: double, Found: INT32
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:278)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:561)
	at org.apache.spark.sql.c

In [ ]:
from pyspark.sql.types import IntegerType

def month_classification(name: str) -> DataFrame:
    return F.split(col(name), " ", )[0].alias("datetime")
    

col_name = "Trip_Pickup_DateTime"
# Replace 'data' with your DataFrame
null_check = data.withColumn(
    "year",
    F.when(col(col_name) != "null",
           F.year(month_classification(col_name)).cast(IntegerType())).otherwise(None)
).withColumn(
    "month",
    F.when(col(col_name) != "null",
           F.month(month_classification(col_name)).cast(IntegerType())).otherwise(None)
).withColumn(
    "day",
    F.when(col(col_name) != "null",
           F.dayofmonth(month_classification(col_name)).cast(IntegerType())).otherwise(None)
).select(
    col("Trip_Pickup_DateTime"),
    col("year"),
    col("month"),
    col("day"),
).groupBy(col("month"), col("day")).agg(F.count("*").alias("count")).cache()

In [ ]:
na_delect = null_check.filter(col("month").isNotNull())
filtered = null_check.select(
    "*"
).orderBy(
    F.asc(col("month")), 
    F.asc(col("day")),
).show()

In [ ]:
spark.stop()